# Capstone Project

## Battle of neighbourhood

## 1.Introduction
This project is for looking at the prevalent businesses in the top two cities in India. Mumbai and New Delhi are the metropolitan cities in India. For any new chain of businesses that want to spread their business in India, These are the prime cities. Based on the experience gathered in these cities, similar pattern can be adopted for other cities such as Kolkata, Chennai, Hyderabad, Bangalore, Pune etc.

## 2.Business Problem
To analyze the business opportunity, current pattern of the businesses have to be understood. Most popular businesses can provide the good bottom line due to existing demand. They need the right location.It is more profitable from the start if demand already exists.
The opportunity of starting a specialized area in most prevalent business at the right location has to be assessed. 

### 2.1 Target audience

 a. Businessmen wants to understand most popular business in metro India
 
 b. People visiting Delhi Mumbai

## 3.Data
The data has been taken from GeoNames Postal Code files. They have provided the IN.txt file that contains all post codes of India. The following is the structure of the file The data format is tab-delimited text in utf8 encoding, with the following fields :

The data format is tab-delimited text in utf8 encoding, with the following fields :

**country code**      : iso country code, 2 characters

**postal code**       : varchar(20)

**place name**        : varchar(180)

**admin name1**       : 1. order subdivision (state) varchar(100)

**admin code1**       : 1. order subdivision (state) varchar(20)

**admin name2**       : 2. order subdivision (county/province) varchar(100)

**admin code2**       : 2. order subdivision (county/province) varchar(20)

**admin name3**       : 3. order subdivision (community) varchar(100)

**admin code3**       : 3. order subdivision (community) varchar(20)

**latitude**          : estimated latitude (wgs84)

**longitude**         : estimated longitude (wgs84)

**accuracy**          : accuracy of lat/lng from 1=estimated, 4=geonameid, 6=centroid of addresses or shape


In [1]:
## Importing the required libraries

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.11

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Libraries imported.


The input file doesn't have header. It is also comma delimited. 
The file is opened and header is changed.
Not required columns are removed.

In [2]:
PIN_list = pd.read_csv("IN.txt",sep='\t')
PIN_list.head()

,IN,744301,Kakana,Andaman & Nicobar Islands,01,Nicobar,638,Carnicobar,Unnamed: 8,9.1167,92.8,4
0,IN,744301,Sawai,Andaman & Nicobar Islands,1,Nicobar,638.0,Carnicobar,NaN,7.5166,93.6031,4
1,IN,744301,Mus,Andaman & Nicobar Islands,1,Nicobar,638.0,Carnicobar,NaN,9.2333,92.7833,4
2,IN,744301,Lapathy,Andaman & Nicobar Islands,1,Nicobar,638.0,Carnicobar,NaN,9.1833,92.7667,3
3,IN,744301,Carnicobar,Andaman & Nicobar Islands,1,Nicobar,638.0,Carnicobar,NaN,9.1833,92.7667,3
4,IN,744302,Little Nicobar,Andaman & Nicobar Islands,1,Nicobar,638.0,Nancowrie,NaN,9.1833,92.7667,1


In [3]:
PIN_list.rename( columns = {'744301': 'PIN', 'Kakana' : 'Place', 'Andaman & Nicobar Islands': 'area' , '9.1167': 'latitude', '92.8' : 'longitude', '4' : 'accuracy', 'Carnicobar': 'area2', 'Nicobar': 'region'  }, inplace=True)
PIN_list = PIN_list.drop(["01", "638", "Unnamed: 8", "IN"], axis=1)

Two separate dataframes are created
1. for Mumbai
2. for New Delhi


In [4]:
Mumbai_df = PIN_list[PIN_list["area2"] == "Mumbai"]

Mumbai_df.head()

,PIN,Place,area,region,area2,latitude,longitude,accuracy
42850,400001,Mumbai G.P.O.,Maharashtra,Mumbai,Mumbai,18.9474,72.8138,1
42851,400001,Bazargate,Maharashtra,Mumbai,Mumbai,18.9474,72.8138,1
42852,400001,M.P.T.,Maharashtra,Mumbai,Mumbai,18.9474,72.8138,1
42853,400001,Tajmahal,Maharashtra,Mumbai,Mumbai,18.9474,72.8138,1
42854,400001,Town Hall (Mumbai),Maharashtra,Mumbai,Mumbai,18.9474,72.8138,1


In [5]:
Delhi_df = PIN_list[PIN_list["region"] == "New Delhi"]

Delhi_df.head()

,PIN,Place,area,region,area2,latitude,longitude,accuracy
14927,110001,New Delhi G.P.O.,Delhi,New Delhi,New Delhi,28.6369,77.2183,3
14928,110002,Civic Centre,Delhi,New Delhi,New Delhi Central,28.6453,77.2456,3
14929,110020,F F C Okhla,Delhi,New Delhi,New Delhi,28.5345,77.2779,3
14930,110029,Nauroji Nagar,Delhi,New Delhi,New Delhi,28.5650,77.1972,3
14931,110043,Gopal Nagar,Delhi,New Delhi,Delhi,28.6109,77.2140,4


In [6]:
Delhi_df.sort_values("region", inplace = True) 


/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [7]:
Delhi_df[['region','latitude', 'longitude']].drop_duplicates()
Mumbai_df[['region','latitude', 'longitude']].drop_duplicates()

,region,latitude,longitude
42850,Mumbai,18.9474,72.8138
42856,Mumbai,18.9750,72.8258
42863,Mumbai,18.9500,72.8167
42869,Mumbai,18.9069,72.8106
42874,Mumbai,18.9596,72.8496
42875,Mumbai,18.9636,72.8193
42876,Mumbai,18.9606,72.8220
42878,Mumbai,18.9667,72.8167
42893,Mumbai,18.9833,72.8333
42898,Mumbai,19.0000,72.8333


In [8]:
latitude = 28.6369
longitude = 77.2183
map_delhi = folium.Map(location=[latitude, longitude], zoom_start=8)

# add markers to map
for lat, lng, local in zip(Delhi_df['latitude'], Delhi_df['longitude'], Delhi_df['region']):
    label = '{}'.format(local)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_delhi)  

print ( "Different postal codes in New Delhi")
map_delhi

Different postal codes in New Delhi


Parameters are created for calling Foursquare api

In [9]:
import requests

CLIENT_ID = 'KZPW3HHRI0NF3BGFVAXEDK2INGBNUYGLV1ABPBIPRNXDOTPN' # your Foursquare ID
CLIENT_SECRET = 'SQMKGW3WH3BUCCDPM43KVOIEAMGZ21OXGBVFJ5TDTXQRWERH' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: KZPW3HHRI0NF3BGFVAXEDK2INGBNUYGLV1ABPBIPRNXDOTPN
CLIENT_SECRET:SQMKGW3WH3BUCCDPM43KVOIEAMGZ21OXGBVFJ5TDTXQRWERH


In [13]:
####function below is taken from the internet.After calling the four square api, the json data is read 

def locations(names, latitudes, longitudes, radius=1500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit=100'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng,
            radius)

        
        # connect and call url
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
            # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Finding the nearby places of interest in New Delhi area using the Foursquare api and then finding the venues having the highest number

In [14]:

Delhi_places=locations(Delhi_df['Place'], Delhi_df['latitude'], Delhi_df['longitude'])

In [15]:
tempD = Delhi_places.groupby("Venue Category").count().reset_index().rename(columns={'State': 'Count'})

In [16]:
tempD.sort_values("Neighborhood", axis = 0, ascending = False).head(20)

,Venue Category,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude
38,Indian Restaurant,41,41,41,41,41,41
12,Café,22,22,22,22,22,22
34,Hotel,22,22,22,22,22,22
66,Restaurant,14,14,14,14,14,14
16,Coffee Shop,12,12,12,12,12,12
8,Bar,9,9,9,9,9,9
45,Lounge,8,8,8,8,8,8
4,Asian Restaurant,6,6,6,6,6,6
13,Chinese Restaurant,6,6,6,6,6,6
61,Pizza Place,6,6,6,6,6,6


Finding the nearby places of interest in Mumbai area using the Foursquare api and then finding the venues having the highest number

In [17]:
Mumbai_places=locations(Mumbai_df['Place'], Mumbai_df['latitude'], Mumbai_df['longitude'])

In [18]:
tempB = Mumbai_places.groupby("Venue Category").count().reset_index().rename(columns={'State': 'Count'})

In [19]:
tempB.sort_values("Neighborhood", axis = 0, ascending = False).head(20)

,Venue Category,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude
86,Indian Restaurant,1050,1050,1050,1050,1050,1050
55,Fast Food Restaurant,376,376,376,376,376,376
35,Coffee Shop,376,376,376,376,376,376
29,Café,373,373,373,373,373,373
131,Restaurant,295,295,295,295,295,295
13,Bakery,210,210,210,210,210,210
31,Chinese Restaurant,185,185,185,185,185,185
85,Ice Cream Shop,183,183,183,183,183,183
14,Bar,181,181,181,181,181,181
140,Snack Place,159,159,159,159,159,159


#### Now let's look at the businesses that have very few numbers

These businesses have just one number in the city. 

In [20]:
tempD.sort_values("Neighborhood", axis = 0, ascending = True).head(20)

,Venue Category,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude
0,ATM,1,1,1,1,1,1
42,Juice Bar,1,1,1,1,1,1
89,Wings Joint,1,1,1,1,1,1
48,Metro Station,1,1,1,1,1,1
49,Miscellaneous Shop,1,1,1,1,1,1
50,Molecular Gastronomy Restaurant,1,1,1,1,1,1
52,Mosque,1,1,1,1,1,1
53,Motel,1,1,1,1,1,1
54,Movie Theater,1,1,1,1,1,1
55,Museum,1,1,1,1,1,1


In [21]:
tempB.sort_values("Neighborhood", axis = 0, ascending = True).head(20)

,Venue Category,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude
46,Design Studio,1,1,1,1,1,1
2,American Restaurant,1,1,1,1,1,1
7,Arts & Crafts Store,1,1,1,1,1,1
63,Food Court,1,1,1,1,1,1
130,Resort,1,1,1,1,1,1
12,Bagel Shop,1,1,1,1,1,1
43,Dance Studio,1,1,1,1,1,1
94,Light Rail Station,1,1,1,1,1,1
0,ATM,2,2,2,2,2,2
48,Dim Sum Restaurant,2,2,2,2,2,2


### Looking at only the eateries and restaurants

Let us look at only the category of Food/ resaurants in the two cities

In [22]:
####function below is taken from the internet.After calling the four square api, the pa

def Food_locations(names, latitudes, longitudes, radius=1500):
    category= '4d4b7105d754a06374d81259'
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        
            
        # create the API request URL
        #url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit=100'.format(
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit=100'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng,
            category,
            radius)

        
        # connect and call url
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
            # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Top Food place categories in New Delhi

In [23]:
Delhi_food_places=Food_locations(Delhi_df['Place'], Delhi_df['latitude'], Delhi_df['longitude'])

In [24]:
tempDFood = Delhi_food_places.groupby("Venue Category").count().reset_index().rename(columns={'State': 'Count'})

In [25]:
tempDFood.sort_values("Neighborhood", axis = 0, ascending = False).head(20)

,Venue Category,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude
18,Indian Restaurant,54,54,54,54,54,54
7,Café,27,27,27,27,27,27
33,Restaurant,21,21,21,21,21,21
13,Fast Food Restaurant,18,18,18,18,18,18
8,Chinese Restaurant,11,11,11,11,11,11
31,Pizza Place,11,11,11,11,11,11
1,Asian Restaurant,8,8,8,8,8,8
37,Snack Place,7,7,7,7,7,7
19,Italian Restaurant,7,7,7,7,7,7
6,Breakfast Spot,6,6,6,6,6,6


Top Food place categories in Mumbai

In [ ]:
Mumbai_food_places=Food_locations(Mumbai_df['Place'], Mumbai_df['latitude'], Mumbai_df['longitude'])

In [ ]:
tempBFood = Mumbai_food_places.groupby("Venue Category").count().reset_index().rename(columns={'State': 'Count'})

In [ ]:
tempBFood.sort_values("Neighborhood", axis = 0, ascending = False).head(20)

In [ ]:
import matplotlib.pyplot as plt

ax = Mumbai_food_places['Venue Category'].value_counts().plot(kind='bar',
                                    figsize=(14,8),
                                    title="Food business in Mumbai")
ax.set_xlabel("Food joint category")
ax.set_ylabel("Frequency")
plt.show()

## Observations


1. The data shows that the restaurants, Eateries, bars are the top businesses in Mumbai. The same pattern is seen in New Delhi. Indian restaurants, Fast Food, Chinese restaurants, Coffee, ice cream shops, juice bar are at the top.

2. Portugese restaurants, American restaurants are few in numbers. There is oppotunity for such restaurants as speciality ones. The same is true for South American, Korean and Spanish restaurants.  

3. In New Delhi, Museum also appeared in top 5 as there are many museums in New Delhi. While in Mumbai, it is very low in the venue category.

4. The categorization available at the Foursquare database is not refined as there are separate categories for 'New American Restaurant' and 'American Restaurant', it is difficult to grasp the difference. Similarly, there is one item falling under category 'Road'. 



In [ ]:
Mumbai_food_places.head()

In [ ]:
vc = Mumbai_food_places["Venue Category"].value_counts()
u  = [i not in set(vc[vc>4].index) for i in Mumbai_food_places["Venue Category"]]
Mum_Food_speciality = Mumbai_food_places[u]

In [ ]:
Mum_Food_speciality

In [ ]:
import matplotlib.pyplot as plt

ax = Delhi_food_places['Venue Category'].value_counts().plot(kind='bar',
                                    figsize=(14,8),
                                    title="Food business in Delhi")
ax.set_xlabel("Food joint category")
ax.set_ylabel("Frequency")
plt.show()

In [ ]:
vc = Delhi_food_places["Venue Category"].value_counts()
u  = [i not in set(vc[vc>4].index) for i in Delhi_food_places["Venue Category"]]
NDelhi_Food_speciality = Delhi_food_places[u]

In [ ]:
latitude = 18.9274
longitude = 72.8241

map_Mumbai = folium.Map(location=[latitude, longitude], zoom_start=8)

# add markers to map
for lat, lng, local in zip(Mum_Food_speciality['Venue Latitude'], Mum_Food_speciality['Venue Longitude'], Mum_Food_speciality['Venue Category']):
    label = '{}'.format(local)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_Mumbai)  

print ( "Existing less prevalent speciality restaurants in Mumbai")
map_Mumbai

In [ ]:
latitude = 28.6369
longitude = 77.2183

map_Delhi = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, local in zip(NDelhi_Food_speciality['Venue Latitude'], NDelhi_Food_speciality['Venue Longitude'], NDelhi_Food_speciality['Venue Category']):
    label = '{}'.format(local)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_Delhi)  

print ( "Existing less prevalent speciality restaurants in New Delhi")
map_Delhi

## Recommendations

The existing speciality restaurants are mainly in South Mumbai and Bandra area.
In New Delhi, the speciality restaurants are mainly in Paharganj and Cannaught Place areas.

These are the areas where there are more tourists. 
It is recommended that the new speciality restaurant is opened in the same area.

One other reason is that these areas are the areas with affluent people of the city.